## Fitting a diagonal covariance Gaussian mixture model to text data

In a previous assignment, we explored k-means clustering for a high-dimensional Wikipedia dataset. We can also model this data with a mixture of Gaussians, though with increasing dimension we run into two important issues associated with using a full covariance matrix for each component.
 * Computational cost becomes prohibitive in high dimensions: score calculations have complexity cubic in the number of dimensions M if the Gaussian has a full covariance matrix.
 * A model with many parameters require more data: bserve that a full covariance matrix for an M-dimensional Gaussian will have M(M+1)/2 parameters to fit. With the number of parameters growing roughly as the square of the dimension, it may quickly become impossible to find a sufficient amount of data to make good inferences.

Both of these issues are avoided if we require the covariance matrix of each component to be diagonal, as then it has only M parameters to fit and the score computation decomposes into M univariate score calculations. Recall from the lecture that the M-step for the full covariance is:

\begin{align*}
\hat{\Sigma}_k &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_i-\hat{\mu}_k)(x_i - \hat{\mu}_k)^T
\end{align*}

Note that this is a square matrix with M rows and M columns, and the above equation implies that the (v, w) element is computed by

\begin{align*}
\hat{\Sigma}_{k, v, w} &= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})(x_{iw} - \hat{\mu}_{kw})
\end{align*}

When we assume that this is a diagonal matrix, then non-diagonal elements are assumed to be zero and we only need to compute each of the M elements along the diagonal independently using the following equation. 

\begin{align*}
\hat{\sigma}^2_{k, v} &= \hat{\Sigma}_{k, v, v}  \\
&= \frac{1}{N_k^{soft}} \sum_{i=1}^N r_{ik} (x_{iv}-\hat{\mu}_{kv})^2
\end{align*}

In this section, we will use an EM implementation to fit a Gaussian mixture model with **diagonal** covariances to a subset of the Wikipedia dataset. The implementation uses the above equation to compute each variance term. 

We'll begin by importing the dataset and coming up with a useful representation for each article. After running our algorithm on the data, we will explore the output to see whether we can give a meaningful interpretation to the fitted parameters in our model.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import necessary packages

In [1]:
import graphlab

We also have a Python file containing implementations for several functions that will be used during the course of this assignment.

In [2]:
from em_utilities import *

## Load Wikipedia data and extract TF-IDF features

Load Wikipedia data and transform each of the first 5000 document into a TF-IDF representation.

In [3]:
wiki = graphlab.SFrame('people_wiki.gl/').head(5000)
wiki['tf_idf'] = graphlab.text_analytics.tf_idf(wiki['text'])

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1467696128.log
INFO:graphlab.cython.cy_server:GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1467696128.log


This non-commercial license of GraphLab Create is assigned to suneel.marthi@gmail.com and will expire on December 24, 2016. For commercial licensing options, visit https://dato.com/buy/.


Using a utility we provide, we will create a sparse matrix representation of the documents. This is the same utility function you used during the previous assignment on k-means with text data.

In [21]:
tf_idf, map_index_to_word = sframe_to_scipy(wiki, 'tf_idf')
print map_index_to_word

+---------+--------------+-------+
| feature |   category   | index |
+---------+--------------+-------+
| feature |  conflating  |   0   |
| feature |   diamono    |   1   |
| feature |   bailouts   |   2   |
| feature | electionruss |   3   |
| feature |   maywoods   |   4   |
| feature |   feduring   |   5   |
| feature |  spiderbait  |   6   |
| feature |     mcin     |   7   |
| feature |  sumiswald   |   8   |
| feature |    quinta    |   9   |
+---------+--------------+-------+
[100282 rows x 3 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


As in the previous assignment, we will normalize each document's TF-IDF vector to be a unit vector. 

In [22]:
tf_idf = normalize(tf_idf)
print tf_idf[0:10]

  (0, 1069)	0.117087926312
  (0, 5547)	0.58543963156
  (0, 16021)	0.117087926312
  (0, 21114)	0.117087926312
  (0, 29521)	0.117087926312
  (0, 39798)	0.117087926312
  (0, 67628)	0.107559063437
  (0, 81889)	0.101985035981
  (0, 83752)	0.0980302005621
  (0, 84782)	0.0980302005621
  (0, 86623)	0.0949625918903
  (0, 87290)	0.0949625918903
  (0, 87908)	0.277368519317
  (0, 88414)	0.0924561731057
  (0, 89402)	0.0903370262183
  (0, 90509)	0.0885013376871
  (0, 90878)	0.0885013376871
  (0, 92764)	0.0829273102307
  (0, 93352)	0.0818269436607
  (0, 94018)	0.0798597015589
  (0, 94280)	0.0789724748121
  (0, 94344)	0.0789724748121
  (0, 95534)	0.0745946139176
  (0, 95612)	0.147967049452
  (0, 95836)	0.0733984473557
  :	:
  (9, 100191)	0.0861422853057
  (9, 100198)	0.0420890606053
  (9, 100205)	0.0202116876236
  (9, 100213)	0.0989526498261
  (9, 100227)	0.0184624536201
  (9, 100229)	0.0183558319253
  (9, 100250)	0.014302748163
  (9, 100253)	0.0120097092568
  (9, 100257)	0.0551241881407
  (9, 100260)

We can check that the length (Euclidean norm) of each row is now 1.0, as expected.

In [6]:
for i in range(5):
    doc = tf_idf[i]
    print(np.linalg.norm(doc.todense()))

1.0
1.0
1.0
1.0
1.0


## EM in high dimensions

EM for high-dimensional data requires some special treatment:
 * E step and M step must be vectorized as much as possible, as explicit loops are dreadfully slow in Python.
 * All operations must be cast in terms of sparse matrix operations, to take advantage of computational savings enabled by sparsity of data.
 * Initially, some words may be entirely absent from a cluster, causing the M step to produce zero mean and variance for those words.  This means any data point with one of those words will have 0 probability of being assigned to that cluster since the cluster allows for no variability (0 variance) around that count being 0 (0 mean). Since there is a small chance for those words to later appear in the cluster, we instead assign a small positive variance (~1e-10). Doing so also prevents numerical overflow.
 
We provide the complete implementation for you in the file `em_utilities.py`. For those who are interested, you can read through the code to see how the sparse matrix implementation differs from the previous assignment. 

You are expected to answer some quiz questions using the results of clustering.

**Initializing mean parameters using k-means**

Recall from the lectures that EM for Gaussian mixtures is very sensitive to the choice of initial means. With a bad initial set of means, EM may produce clusters that span a large area and are mostly overlapping. To eliminate such bad outcomes, we first produce a suitable set of initial means by using the cluster centers from running k-means.  That is, we first run k-means and then take the final set of means from the converged solution as the initial means in our EM algorithm.

In [47]:
from sklearn.cluster import KMeans

np.random.seed(5)
num_clusters = 25

# Use scikit-learn's k-means to simplify workflow
kmeans_model = KMeans(n_clusters=num_clusters, n_init=5, max_iter=400, random_state=1, n_jobs=-1)
kmeans_model.fit(tf_idf)
centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_

means = [centroid for centroid in centroids]
print len(means[0])
print means[0][np.argsort(means[0])][100000:100282]

100282
[ 0.00418968  0.00420258  0.00420436  0.00422626  0.00422786  0.00422916
  0.00422946  0.00423331  0.0042378   0.00424811  0.00424949  0.00425002
  0.00425321  0.00426325  0.00426454  0.00426703  0.00427902  0.00429599
  0.00430408  0.00431684  0.00433565  0.00433794  0.00434281  0.00434861
  0.0043582   0.00436776  0.00437726  0.00440398  0.00441298  0.00441321
  0.00441327  0.00443781  0.00445308  0.00446628  0.00446802  0.00448193
  0.00449216  0.00449399  0.00450011  0.00450937  0.00451453  0.0045162
  0.0045169   0.00451757  0.00453299  0.00453322  0.00454265  0.00456207
  0.00456925  0.00456959  0.00458379  0.00458667  0.00460128  0.00461884
  0.00466576  0.00469142  0.00469348  0.00469702  0.00473195  0.00474032
  0.00474742  0.00476935  0.00480575  0.00480848  0.00482027  0.00483526
  0.00483957  0.00484524  0.00487456  0.00488712  0.00491779  0.00493892
  0.00499029  0.00499423  0.00502721  0.00503224  0.00505576  0.00506654
  0.00509566  0.00510134  0.00510311  0.00522

**Initializing cluster weights**

We will initialize each cluster weight to be the proportion of documents assigned to that cluster by k-means above.

In [13]:
num_docs = tf_idf.shape[0]
weights = []
for i in xrange(num_clusters):
    # Compute the number of data points assigned to cluster i:
    num_assigned = len(cluster_assignment[cluster_assignment == i])
    w = float(num_assigned) / num_docs
    weights.append(w)

**Initializing covariances**

To initialize our covariance parameters, we compute $\hat{\sigma}_{k, j}^2 = \sum_{i=1}^{N}(x_{i,j} - \hat{\mu}_{k, j})^2$ for each feature $j$.  For features with really tiny variances, we assign 1e-8 instead to prevent numerical instability. We do this computation in a vectorized fashion in the following code block.

In [18]:
covs = []
for i in xrange(num_clusters):
    member_rows = tf_idf[cluster_assignment==i]
    cov = (member_rows.power(2) - 2*member_rows.dot(diag(means[i]))).sum(axis=0).A1 / member_rows.shape[0] \
          + means[i]**2
    cov[cov < 1e-8] = 1e-8
    covs.append(cov)

**Running EM**

Now that we have initialized all of our parameters, run EM.

In [19]:
out = EM_for_high_dimension(tf_idf, means, covs, weights, cov_smoothing=1e-10)

In [20]:
out['loglik']

[3855847476.7012835, 4844053202.46348, 4844053202.46348]

## Interpret clustering results

In contrast to k-means, EM is able to explicitly model clusters of varying sizes and proportions. The relative magnitude of variances in the word dimensions tell us much about the nature of the clusters.

Write yourself a cluster visualizer as follows.  Examining each cluster's mean vector, list the 5 words with the largest mean values (5 most common words in the cluster). For each word, also include the associated variance parameter (diagonal element of the covariance matrix). 

A sample output may be:
```
==========================================================
Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
football    1.08e-01    8.64e-03
season      5.80e-02    2.93e-03
club        4.48e-02    1.99e-03
league      3.94e-02    1.08e-03
played      3.83e-02    8.45e-04
...
```

In [58]:
# Fill in the blanks
def visualize_EM_clusters(tf_idf, means, covs, map_index_to_word):
    print('')
    print('==========================================================')

    num_clusters = len(means)
    for c in xrange(num_clusters):
        print('Cluster {0:d}: Largest mean parameters in cluster '.format(c))
        print('\n{0: <12}{1: <12}{2: <12}'.format('Word', 'Mean', 'Variance'))
        
        # The k'th element of sorted_word_ids should be the index of the word 
        # that has the k'th-largest value in the cluster mean. Hint: Use np.argsort().
        sorted_word_ids = np.argsort(means[c]) # YOUR CODE HERE

        for i in sorted_word_ids[:5]:
            print '{0: <12}{1:<10.2e}{2:10.2e}'.format(map_index_to_word['category'][i], 
                                                       means[c][i],
                                                       covs[c][i])
        print '\n=========================================================='

In [59]:
'''By EM'''
visualize_EM_clusters(tf_idf, out['means'], out['covs'], map_index_to_word)


Cluster 0: Largest mean parameters in cluster 

Word        Mean        Variance    
[    0 62037 62036 62035 62034]
conflating  0.00e+00    1.00e-10
cuestin     0.00e+00    1.00e-10
proteccin   0.00e+00    1.00e-10
crayonswarren0.00e+00    1.00e-10
strothers   0.00e+00    1.00e-10

Cluster 1: Largest mean parameters in cluster 

Word        Mean        Variance    
[    0 62704 62703 62702 62701]
conflating  0.00e+00    1.00e-10
clrnan      0.00e+00    1.00e-10
societymustafa0.00e+00    1.00e-10
heartpounding0.00e+00    1.00e-10
preethi     0.00e+00    1.00e-10

Cluster 2: Largest mean parameters in cluster 

Word        Mean        Variance    
[    0 61824 61823 61822 61821]
conflating  0.00e+00    1.00e-10
farnhams    0.00e+00    1.00e-10
gyxa        0.00e+00    1.00e-10
villainswhen0.00e+00    1.00e-10
awax        0.00e+00    1.00e-10

Cluster 3: Largest mean parameters in cluster 

Word        Mean        Variance    
[    0 64832 64831 64830 64829]
conflating  0.00e+00    1.00e

**Quiz Question**. Select all the topics that have a cluster in the model created above. [multiple choice]

## Comparing to random initialization

Create variables for randomly initializing the EM algorithm. Complete the following code block.

In [ ]:
np.random.seed(5)
num_clusters = len(means)
num_docs, num_words = tf_idf.shape

random_means = []
random_covs = []
random_weights = []

for k in range(num_clusters):
    
    # Create a numpy array of length num_words with random normally distributed values.
    # Use the standard univariate normal distribution (mean 0, variance 1).
    # YOUR CODE HERE
    mean = ...
    
    # Create a numpy array of length num_words with random values uniformly distributed between 1 and 5.
    # YOUR CODE HERE
    cov = ...

    # Initially give each cluster equal weight.
    # YOUR CODE HERE
    weight = ...
    
    random_means.append(mean)
    random_covs.append(cov)
    random_weights.append(weight)

**Quiz Question**: Try fitting EM with the random initial parameters you created above. (Use `cov_smoothing=1e-5`.) Store the result to `out_random_init`. What is the final loglikelihood that the algorithm converges to? 

**Quiz Question:** Is the final loglikelihood larger or smaller than the final loglikelihood we obtained above when initializing EM with the results from running k-means?

**Quiz Question**: For the above model, `out_random_init`, use the `visualize_EM_clusters` method you created above. Are the clusters more or less interpretable than the ones found after initializing using k-means?

In [ ]:
# YOUR CODE HERE. Use visualize_EM_clusters, which will require you to pass in tf_idf and map_index_to_word.
...

## Takeaway

In this assignment we were able to apply the EM algorithm to a mixture of Gaussians model of text data. This was made possible by modifying the model to assume a diagonal covariance for each cluster, and by modifying the implementation to use a sparse matrix representation. In the second part you explored the role of k-means initialization on the convergence of the model as well as the interpretability of the clusters.